In [20]:
import pandas as pd
#from tensorflow.keras.models import load_model
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load df, X & y

In [3]:
df = pd.read_csv('/workspaces/spam/df.csv')

# Load Model

In [4]:
loaded_model = load_model('16D_20epochs.keras')  # Load the model

In [5]:
loaded_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 59, 16)         │       320,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 16)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 960,917 (3.67 MB)

 Trainable params: 320,305 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 640,612 (2.44 MB)

# Load Custom Tokenizer

In [6]:
# To load the tokenizer next time
with open('/workspaces/spam/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Transfer learning with gnews

In [84]:
#! pip install tensorflow-hub
#! pip install tensorflow_text

In [21]:
import tensorflow_hub as hub
import tensorflow_text as text
#from tensorflow.keras.layers import Input, Dense

In [23]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")
# encoder_inputs = preprocessor(text_input)
# encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/bert-en-uncased-l-10-h-128-a-2/versions/2", trainable=True)
# outputs = encoder(encoder_inputs)
# pooled_output = outputs["pooled_output"]      # [batch_size, 128].
# sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 128].


In [17]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

#hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], dtype=tf.string, trainable=True)
hub_layer = hub.keras_layer.KerasLayer(model, output_shape=[20], input_shape=[], dtype=tf.string, trainable=True)
#old layer -> tf.keras.layers.Embedding(num_words+1, 16, input_shape=[input_shape,], name="embedding"),

## test

In [18]:
# Convert your input texts to a TensorFlow tensor and ensure there's a batch dimension
texts = ["I love sushi", "why do I love"]
#text2 = [df['v2_clean'].values[0], df['v2_clean'].values[1]]
text_tensor = tf.constant(texts)  # This creates a tensor with the correct shape

# Now pass the tensor to the hub_layer
embeddings = hub_layer(text_tensor)

print(embeddings)

tf.Tensor(
[[ 0.9600803  -0.16374904  0.07920823 -0.03939129 -2.4352     -0.35645273
  -1.0167998   1.0247008   1.9892384   0.59509754 -1.7162676   0.6529031
   0.16242684  0.46056685 -2.0652692  -1.0301673   1.108994    0.22146793
  -1.0159421  -0.9394092 ]
 [ 0.79445785 -1.5408051   1.3680998   0.1702975  -3.004923   -2.1498508
  -1.158673    1.2076448   1.8992825   1.208827   -1.1920041  -0.73729205
   1.1840041   0.7548168  -2.2231488  -0.24037983  2.2688918  -0.02528287
  -0.8088422  -0.770233  ]], shape=(2, 20), dtype=float32)


In [7]:
# Assuming 'df' is your DataFrame
texts = df['v2_clean'].values
labels = df['encoding'].values

## model

In [11]:
# Use Keras 2.
version_fn = getattr(tf.keras, "version", None)

In [24]:
version_fn()

'3.0.5'

In [13]:
import tf_keras as keras

In [19]:
model = tf.keras.Sequential([
    hub_layer,
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7f22e63807f0> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [13]:
model.summary()

AttributeError: 'str' object has no attribute 'summary'